# Prepare Mask Data Delete Red Fill Black by moriitkys (2021/02)
LICENSE : MIT

<img src="Documents/imgs/AugDataSet_structure_all.png" width="400">

# Delete Red Pixels
まず生画像からセグメンテーションラベル用に選ぶ特定の色を削除する。それがDelete Red Pixelsで実行される。  
First remove the specific color you choose for the segmentation label from the raw image. It runs in Delete Red Pixels.   

今回塗りつぶす色はwindowsのペイントのパレットの赤い色BGR=（36, 28, 237）。そのあと、インスタンスセグメンテーション用に段階的に色を塗り分ける作業もある（別のノートブック）。  
The color to be filled this time is the red color BGR = (36, 28, 237) of the paint palette of windows. Then there's the work of stepping through the colors for instance segmentation (another notebook).   

<img src="Documents/imgs/prepare_mask_data_drfb_deletered.png" width="300">

In [4]:
# ----- Prepare Mask Data - Delete Red -----
import sys
import shutil
import glob
import os
import csv

import numpy as np
import random
import matplotlib.pyplot as plt

import PIL
from PIL import Image
import cv2

from pathlib import Path
import shutil

from sklearn.model_selection import train_test_split

import mylib.create_panel_for_selectfolder as create_panel
import mylib.config as config
config_param = config.ConfigParameters()

# ------ Setting panels ------
import tkinter
from tkinter import messagebox

root1 = tkinter.Tk()
# You can't use root1 = tkinter.withdraw()
# ----- Yes・No Confirm Target Dir -----
text_message_window = "Delete Redは通常使いません。\n"+\
                        "元画像から赤い画素を削除した画像を生成しますか？\n"+\
                        "Delete Red usually aren't used.\n"+\
                        "Do you want to generate an image with the red pixels removed from the original image?"
                        
ret = messagebox.askyesno('Confirm Target Dir', text_message_window)

root1.destroy()

if not ret:
    sys.exit()

# Searching extention : "_c_raw.png"
key_for_searching_foldername = config_param.key_pmd_drfb_drp
setting_panel = create_panel.CreatePanelForSelectFolder(key_for_searching_foldername)
setting_panel.create_buttons()#If you push "start", exit this line.

list_selected_folders = setting_panel.list_selected_folders
print("list_selected_folders", list_selected_folders)
#print(setting_panel.dict_object_class_number)

for selected_folder in list_selected_folders:
    output_folder = selected_folder
    input_files = glob.glob(selected_folder + "/*" + key_for_searching_foldername)
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    
    for input_file in input_files:
        src = cv2.imread(input_file)
        out = src.copy()
        #height, width = src.shape[:2]
        
        red_alternative = [36, 28, 236]# Not config_param.manual_masking_color ([36, 28, 237])
        out[np.where((out == config_param.manual_masking_color).all(axis=2))] = red_alternative
        
        output_file = input_file.replace("_c_raw", "_c_raw_redmask")
        output_file = output_file.replace(selected_folder, selected_folder)
        cv2.imwrite(output_file, out)
        
        #shutil.copyfile(input_file, output_file)
        
# Write the path of list_selected_folders to csv
list_selected_folders = [ [i] for i in list_selected_folders]
f = open('./log/prepare_mask_data_drfb.csv','w')
# https://work-life-enj.com/pyhon-write-csv/
w = csv.writer(f, lineterminator='\n')
w.writerows(list_selected_folders)
#for i in list_selected_folders:
#    w.writerow(i)
f.close()

print("----- Complete Prepare Mask Data - Delete Red -----")

list_selected_folders []
----- Complete Prepare Mask Data - Delete Red -----


# Manual Coloring the area of the object
Mask R-CNNやSemantic segmentationで推定したい領域（ピクセル群）を特定の色で塗りつぶした教師ラベル画像が必要である。  
A ground trueth label image is required in which the area (pixel group) to be estimated by Mask R-CNN or Semantic segmentation is filled with a specific color. 

# Fill Black Pixels
Ground trueth用画像作成のために先ほど赤で塗りつぶした画像の背景を黒で塗りつぶす。赤い色の部分はわかりやすいように白で塗りつぶす。  
To create an image for Ground trueth, fill the background of the image painted in red with black. Fill the red part with white for easy understanding.   
<img src="Documents/imgs/prepare_mask_data_drfb_fillblack.png" width="350">

In [5]:
# ----- Prepare Mask Data - Fill Black -----
import os
import glob
import tkinter
from tkinter import messagebox
import cv2
import numpy as np
import csv
import mylib.create_panel_for_selectfolder as create_panel
import mylib.config as config
import shutil
config_param = config.ConfigParameters()

f = open('./log/prepare_mask_data_drfb.csv', 'r')
reader = csv.reader(f)
#header = next(reader)
#list_selected_folders = []
dict_selected_folders = {}
for row in reader:
    #list_selected_folders.append(row[0])
    row[0] = row[0].replace("\\", "/")
    rslash_1st = row[0].rfind('/')
    rslash_2nd = row[0][:rslash_1st].rfind('/')
    object_class_number_name = row[0][rslash_2nd+1 :]
    dict_selected_folders[object_class_number_name] = row[0]
str_selected_folders = "\n".join(dict_selected_folders)
#print(str_selected_folders)

root1 = tkinter.Tk()
# You can't use root1 = tkinter.withdraw()

# ----- Yes・No Confirm Target Dir -----
text_message_window = "現在のマスク作成対象フォルダは以下が選択されています。\n"+\
                        "The following is selected as the current mask creation target folder.\n"+\
                        "処理対象フォルダを変更しますか。\n"+\
                        "Do you want to change the processing target folder?\n"+\
                        str_selected_folders
                        
ret = messagebox.askyesno('Confirm Target Dir', text_message_window)

root1.destroy()


# Searching extention : "_c_raw_redmask.png"

# ----- [Main Process] Move Manual-Colored Data by using list_selected_folders -----
list_selected_folders = []
key_for_searching_foldername = config_param.key_pmd_drfb_fbp
if ret:
    # ------ Setting panels ------
    setting_panel = create_panel.CreatePanelForSelectFolder(key_for_searching_foldername)
    setting_panel.create_buttons()#If you push "start", exit this line.

    list_selected_folders = setting_panel.list_selected_folders
    print("list_selected_folders", "\n".join(list_selected_folders))
else:
    for key in dict_selected_folders:
        list_selected_folders.append(dict_selected_folders[key])
        print("list_selected_folders", "\n".join(list_selected_folders))

for selected_folder in list_selected_folders:
    output_folder = selected_folder
    input_files = glob.glob(selected_folder + "/*" + key_for_searching_foldername)
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    
    for input_file in input_files:
        src = cv2.imread(input_file)
        out = src.copy()

        white = [255,255,255]
        black = [0,0,0]
        out[np.where((out != config_param.manual_masking_color).all(axis=2))] = black
        out[np.where((out == config_param.manual_masking_color).all(axis=2))] = white
        out[np.where((out != white).all(axis=2))] = black
        
        output_file = input_file.replace("_c_raw_redmask", "_c_raw_whitemask")
        #output_file = output_file.replace(selected_folder, selected_folder)
        cv2.imwrite(output_file, out)
        
        #shutil.copyfile(input_file, output_file)
        
# Write the path of list_selected_folders to csv
list_selected_folders = [ [i] for i in list_selected_folders]
f = open('./log/prepare_mask_data_drfb.csv','w')
# https://work-life-enj.com/pyhon-write-csv/
w = csv.writer(f, lineterminator='\n')
w.writerows(list_selected_folders)
#for i in list_selected_folders:
#    w.writerow(i)
f.close()

print("----- Complete Prepare Mask Data - FillBlack -----")

list_selected_folders 
----- Complete Prepare Mask Data - FillBlack -----
